In [1]:
import gym  # open ai gym
import pybulletgym  # register PyBullet enviroments with open ai gym
import numpy as np
from numpy import linalg as LA

np.set_printoptions(precision=4, suppress=True)

In [2]:
class Conversions():

    def R3Toso3(self, w):
        ''' Convert from R3 to so3'''
        assert isinstance(w, np.ndarray)
        assert w.size == 3
        w = w.reshape((-1,)) #converting to size(3,)
        so3 = np.array([
            [0, -w[2], w[1]],
            [w[2], 0, -w[0]],
            [-w[1], w[0], 0],
        ])
        return so3
    
    def so3ToR3(self, so3):
        ''' Convert from so3 to R3'''
        assert isinstance(so3, np.ndarray)
        assert so3.shape == (3,3)
        
        return np.array([so3[2,1], so3[0,2], so3[1,0]])
    
    def R6Tose3(self, S):
        '''Convert from R6 to se3'''
        assert isinstance(S, np.ndarray)
        assert S.size == 6
        
        S = S.reshape((-1,)) #converting to size(3,)
        w, v = S[0:3], S[3:]
        se3 = np.zeros((4,4))
        se3[0:3, 0:3] = self.R3Toso3(w)
        se3[0:3, 3] = v
        return se3
    
    def se3ToR6(self, V):
        assert isinstance(V, np.ndarray)
        assert V.shape == (4,4)
        
        w = self.so3ToR3(V[0:3, 0:3])
        v = V[0:3, 3]
        V_R6 = np.hstack((w, v))
        
        assert V_R6.shape == (6,)
        return V_R6
    
    def se3ToSE3(self, S, theta):
        '''
        Convert from se3 to SE3
        Note: R and p formula from pg 113, Ch 3, Modern robotics mechanics, planning, control by Kevin Lynch
        
        :param S: Screw axis of joint
        :type S: numpy array
        :param theta: angle of joint
        :type theta: float or int
        '''
        assert isinstance(S, np.ndarray)
        assert isinstance(theta, float) or isinstance(theta, int)
        assert S.size == 6
        
        S = S.reshape((-1,)) #converting to size(6,)
        w, v = S[0:3], S[3:] #extracting w,v from Screw vector
        w_hat = self.R3Toso3(w)
        R = np.eye(3) + np.sin(theta)*w_hat + (1 - np.cos(theta))*w_hat@w_hat
        p = ((theta*np.eye(3)) + \
             ((1 - np.cos(theta))*w_hat) + \
             ((theta - np.sin(theta))*w_hat@w_hat)) @ v
        SE3 = np.zeros((4,4))
        SE3[0:3,0:3] = R
        SE3[0:3,3] = p
        SE3[3,3] = 1.0
        return SE3
    
    def getAdjunct(self, T):
        '''
        Calculates the adjunct matrix for a homogeneous matrix T which belongs to SE3
        
        :param T: A homogeneous matrix belonging to SE3
        :type T: numpy array, 4x4
        :rtype: numpy array, 6x6
        '''
        R, p = T[0:3,0:3], T[0:3,3]
        p_hat = self.R3Toso3(p)
        AdjunctT = np.zeros((6,6))
        AdjunctT[0:3,0:3] = R
        AdjunctT[3:, 0:3] = p_hat @ R
        AdjunctT[3:,3:] = R
        
        assert AdjunctT.shape == (6,6)
        return AdjunctT
    
    def transToRp(self, T):
        '''
        Returns the Rotation matrix and position vector contained in the homogeneous transformation matrix T
        
        :param T: A homogeneous matrix belonging to SE3
        :type T: numpy array, 4x4
        '''
        assert isinstance(T, np.ndarray)
        assert T.shape == (4,4)
        
        return T[0:3, 0:3], T[0:3, 3]
    
    def invOfTrans(self, T):
        '''
        Returns inverse of the homogeneous transformation matrix T
        
        :param T: A homogeneous matrix belonging to SE3
        :type T: numpy array, 4x4
        '''
        assert isinstance(T, np.ndarray)
        assert T.shape == (4,4)
        
        R, p = self.transToRp(T)
        invT = np.zeros((4,4))
        invT[0:3, 0:3] = R.T
        invT[0:3, 3] = -(R.T @ p)
        invT[3, 3] = 1
        
        assert invT.shape == (4,4)
        return invT

    def nearZero(self, z):
        """Determines whether a scalar is small enough to be treated as zero"""
        return abs(z) < 1e-6
    
    def matrixLogOfR(self, R):
        acosinput = (np.trace(R) - 1) / 2.0
        if acosinput >= 1:
            return np.zeros((3, 3))
        elif acosinput <= -1:
            if not self.nearZero(1 + R[2][2]):
                omg = (1.0 / np.sqrt(2 * (1 + R[2][2]))) * np.array([R[0][2], R[1][2], 1 + R[2][2]])
            elif not self.nearZero(1 + R[1][1]):
                omg = (1.0 / np.sqrt(2 * (1 + R[1][1]))) * np.array([R[0][1], 1 + R[1][1], R[2][1]])
            else:
                omg = (1.0 / np.sqrt(2 * (1 + R[0][0]))) * np.array([1 + R[0][0], R[1][0], R[2][0]])
            return self.R3Toso3(np.pi * omg)
        else:
            theta = np.arccos(acosinput)
            return theta / 2.0 / np.sin(theta) * (R - np.array(R).T)

    def matrixLogOfT(self, T):
        
        R, p = self.transToRp(T)
        omgmat = self.matrixLogOfR(R)
        if np.array_equal(omgmat, np.zeros((3, 3))):
            return np.r_[np.c_[np.zeros((3, 3)),
                               [T[0][3], T[1][3], T[2][3]]],
                         [[0, 0, 0, 0]]]
        else:
            theta = np.arccos((np.trace(R) - 1) / 2.0)
            return np.r_[np.c_[omgmat,
                               np.dot(np.eye(3) - omgmat / 2.0 \
                               + (1.0 / theta - 1.0 / np.tan(theta / 2.0) / 2) \
                                  * np.dot(omgmat,omgmat) / theta,[T[0][3],
                                                                   T[1][3],
                                                                   T[2][3]])],
                         [[0, 0, 0, 0]]]


In [72]:
def getState():
    q0, q0_dot = env.unwrapped.robot.central_joint.current_position()
    q1, q1_dot = env.unwrapped.robot.elbow_joint.current_position()
    return [q0,q1], [q0_dot, q1_dot]

def setJointAngles(thetalist):
    assert isinstance(thetalist, list) and len(thetalist) == 2
    assert all([isinstance(q, (int,float)) for q in thetalist])
    
    env.unwrapped.robot.central_joint.reset_position(thetalist[0], 0)
    env.unwrapped.robot.robot_elbom.reset_position(thetalist[1], 0)
    return

def getForwardModel(thetalist):
    '''
    For a 2-dof arm, this function takes joint states (q0,q1) as input and returns the end effector position.
    Units for q0, q1 are radians.
    Forward kinematics is done in world/space frame
    
    :param thetalist: list containing 2 joint angles
    :type thetalist: list
    :rtype: numpy array, 4x4
    '''
    assert isinstance(thetalist, list) and len(thetalist) == 2
    assert all([isinstance(q, (int,float)) for q in thetalist])
    
    l0, l1 = 0.1, 0.11
    M = np.array([
        [1, 0, 0, l0+l1],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ])
    S = np.array([[0, 0, 1, 0, 0, 0],
                  [0, 0, 1, 0, -l0, 0],
                 ]).T
    f = Conversions()
    # Forward Kinematics in world/space frame
    T = np.array(M)
    for i in range(len(thetalist)-1,-1,-1):
        exp_S_theta = f.se3ToSE3(S[:,i], thetalist[i])
        T = exp_S_theta @ T
    return T[0:2,3]


def getJacobian(thetalist):
    '''
    For a 2-dof arm, this function takes joint states (q0,q1) as input and returns the Jacobian.
    Units for q0, q1 are radians.
    Jacobian calculated in space frame/world frame
    
    :param thetalist: list containing 2 joint angles
    :type thetalist: list
    :rtype: numpy array, 2x2
    '''
    assert isinstance(thetalist, list) and len(thetalist) == 2
    assert all([isinstance(q, (int,float)) for q in thetalist])

    l0, l1 = 0.1, 0.11
    theta0, theta1 = thetalist[0], thetalist[1]
    
    term1 = -l0*np.sin(theta0) - l1*np.sin(theta0 + theta1)
    term2 = -l1*np.sin(theta0 + theta1)
    term3 = l0*np.cos(theta0) + l1*np.cos(theta0 + theta1)
    term4 = l0*np.cos(theta0 + theta1)
    
    J = np.array([
        [term1, term2],
        [term3, term4],
    ])
    return J

def getIK(x_d, y_d, initial_guess = np.array([0.3, 0.4]), maxErr = 1e-3):
    '''
    Implements inverse kinematics for the 2-dof arm using an analytical solution
    Ref: https://modernrobotics.northwestern.edu/nu-gm-book-resource/inverse-kinematics-of-open-chains/#department

    :param x_d: Desired x position of end-effector
    :type x_d: int or float
    :param y_d: Desired y position of end-effector
    :type y_d: int or float
    :param initial_guess: initial guess for IK to begin
    :type initial_guess: np.ndarray
    :param maxErr: maximum error allowed between desired and calculated end-effector position
    :type maxErr: float
    :rtype: bool, np.ndarray (if bool == True, it indicates Ik successfully converged)
    '''
    assert isinstance(x_d, (int, float))
    assert isinstance(y_d, (int, float))
    assert isinstance(maxErr, float) and maxErr >= 0
    assert isinstance(initial_guess, np.ndarray) and len(initial_guess) == 2
    
    i, maxIter = 0, 20
    l0, l1 = 0.1, 0.11
    q = initial_guess.copy()
    position_desired = np.array([x_d, y_d])

    while(i<maxIter):
        position = getForwardModel(list(q))
#         print('position = ',position)
        J = getJacobian(list(q))
#         print('J = ', J)
        dx = position_desired - np.array(position)
#         print('dx = ', dx)
        dq = LA.pinv(J) @ dx
#         print('dq = ', dq)
        q_next = q + dq
#         print('q_next = ', q_next)
        position_next = getForwardModel(list(q_next))
#         print('position_next = ', position_next)
        err = LA.norm(position_desired - position_next)
        q = q_next

        if err < maxErr:
            print(i)
            return True, q
        i+=1

    return False, q

In [77]:
success, q = getIK(x_d=0.1, y_d=0.11, initial_guess = np.array([0.3, 0.4]), maxErr = 1e-4)
print(q)

5
[-0.      1.5708]


In [78]:
getForwardModel(list(q))

array([0.1 , 0.11])

In [6]:
env = gym.make ("ReacherPyBulletEnv-v0")
env.reset()
theta, theta_dot = getState()
print(theta, theta_dot)

current_dir=/Users/adnanshahpurwala/anaconda3/envs/ece276c_venv/lib/python3.6/site-packages/pybullet_envs/bullet
options= 
[0.3618124497886881, -2.2590040455940352] [0.0, 0.0]


In [9]:
q_curr, v_curr = getState()
print(q_curr)

AttributeError: 'list' object has no attribute 'shape'